In [1]:
import itertools
import shutil
from multiprocessing import Pool
from src.recommenders import ImprovedRecommender, PopBasedRecommender
from os import cpu_count
from os.path import exists
from sklearn.decomposition import TruncatedSVD
qual_eval_folder = './evaluation'

From the base recommender results, we will now continue to use the ideal combination of methods as a baseline before we make further improvements.
This means we use cosine distance and normalized feature vectors. We also don't use feedback weighting.

The following cell runs the basic recommender and provides more in-depth evaluation and compares it to a popularity-based recommender.

In [2]:
rec = PopBasedRecommender()
rec.generate_recommendations("./data/australian_user_reviews.json", read_max=1000)
print(rec.evaluate(k=5))
dim_red = TruncatedSVD(n_components=250, random_state=500)
rec = ImprovedRecommender("./data/steam_games.json", sparse=True, tfidf='smooth', use_feedback=False, normalize=True, dim_red=dim_red)
rec.generate_recommendations("./data/australian_user_reviews.json", read_max=1000)
rec.evaluate(k=5)

1000it [00:00, 14925.23it/s]


Break reading after 1000 records
Reading 1001 rows.


3536it [00:00, 17551.36it/s]

{'HR@5': 0.9547900968783638, 'nDCG@5': 0.4959111751481147, 'recall@5': 0.036132772974029226, 'ideal_recall@5': 0.09192854361255526, 'nRecall@5': 0.4655005382131325}


32135it [00:01, 17252.34it/s]


Reading 32135 rows.


1000it [00:00, 15151.63it/s]


Break reading after 1000 records
Reading 1001 rows.


c:\Users\Sam\Documents\GitHub\AI-Project\src\recommenders.py:413: RuntimeWarning: Sparse was set to 'True' but dimensionality reduction is used, using dense matrix representation instead.
  warnings.warn("Sparse was set to 'True' but dimensionality reduction is used, using dense matrix representation instead.", RuntimeWarning)
999it [00:43, 22.93it/s]


{'HR@5': 0.5963923337091319,
 'nDCG@5': 0.2062706587989166,
 'recall@5': 0.013222881039268622,
 'ideal_recall@5': 0.09132511265166662,
 'nRecall@5': 0.1927846674182638}